In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.6 MB/s eta 0:00:00


In [8]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 28.0 MB/s eta 0:00:00


In [9]:
!pip install pytesseract

In [10]:
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from glob import glob
from xml.dom import minidom
import random
import easyocr
from ultralytics import YOLO
import yaml
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import pytesseract
from PIL import Image

In [11]:
!ls "/content/drive/MyDrive/Project/annotations"

Cars0.xml    Cars166.xml  Cars231.xml  Cars298.xml  Cars363.xml  Cars429.xml
Cars100.xml  Cars167.xml  Cars232.xml  Cars299.xml  Cars364.xml  Cars42.xml
Cars101.xml  Cars168.xml  Cars233.xml  Cars29.xml   Cars365.xml  Cars430.xml
Cars102.xml  Cars169.xml  Cars234.xml  Cars2.xml    Cars366.xml  Cars431.xml
Cars103.xml  Cars16.xml   Cars235.xml  Cars300.xml  Cars367.xml  Cars432.xml
Cars104.xml  Cars170.xml  Cars236.xml  Cars301.xml  Cars368.xml  Cars43.xml
Cars105.xml  Cars171.xml  Cars237.xml  Cars302.xml  Cars369.xml  Cars44.xml
Cars106.xml  Cars172.xml  Cars238.xml  Cars303.xml  Cars36.xml	 Cars45.xml
Cars107.xml  Cars173.xml  Cars239.xml  Cars304.xml  Cars370.xml  Cars46.xml
Cars108.xml  Cars174.xml  Cars23.xml   Cars305.xml  Cars371.xml  Cars47.xml
Cars109.xml  Cars175.xml  Cars240.xml  Cars306.xml  Cars372.xml  Cars48.xml
Cars10.xml   Cars176.xml  Cars241.xml  Cars307.xml  Cars373.xml  Cars49.xml
Cars110.xml  Cars177.xml  Cars242.xml  Cars308.xml  Cars374.xml  Cars4.xml
Cars111.xm

In [13]:
# -*- coding: utf-8 -*-

import os
from glob import glob
from xml.dom import minidom

# =========================
# CLASS LOOKUP TABLE
# =========================
lut = {}
lut["accessory"] = 0
lut["top"]       = 1
lut["bottom"]    = 2
lut["bag"]       = 3
lut["shoes"]     = 4


# =========================
# COORDINATE CONVERSION
# =========================
def convert_coordinates(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]

    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh

    return (x, y, w, h)


# =========================
# XML  ->  YOLO
# =========================
def convert_xml2yolo(lut, input_path, output_path):

    # safely create output directory (including parents)
    os.makedirs(output_path, exist_ok=True)

    xml_files = glob(f"{input_path}/*.xml")
    print(f"Found {len(xml_files)} XML files")

    for fname in xml_files:

        xmldoc = minidom.parse(fname)
        annot_fname = os.path.splitext(os.path.basename(fname))[0]
        fname_out = os.path.join(output_path, annot_fname + ".txt")

        with open(fname_out, "w") as f:

            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]

            width = int(size.getElementsByTagName('width')[0].firstChild.data)
            height = int(size.getElementsByTagName('height')[0].firstChild.data)

            for item in itemlist:

                # class label
                classid = item.getElementsByTagName('name')[0].firstChild.data.strip()

                if classid in lut:
                    label_str = str(lut[classid])
                else:
                    label_str = "0"   # default fallback

                # bounding box
                xmin = float(item.getElementsByTagName('bndbox')[0].getElementsByTagName('xmin')[0].firstChild.data)
                xmax = float(item.getElementsByTagName('bndbox')[0].getElementsByTagName('xmax')[0].firstChild.data)
                ymin = float(item.getElementsByTagName('bndbox')[0].getElementsByTagName('ymin')[0].firstChild.data)
                ymax = float(item.getElementsByTagName('bndbox')[0].getElementsByTagName('ymax')[0].firstChild.data)

                b = (xmin, xmax, ymin, ymax)
                bb = convert_coordinates((width, height), b)

                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + "\n")

    print("✅ Converting is finished!")


# =========================
# RUN CONVERSION
# =========================

convert_xml2yolo(
    lut,
    input_path = "/content/drive/MyDrive/Project/annotations",
    output_path = "/content/drive/MyDrive/zenodo-license/output/annotations"
)

Found 433 XML files
✅ Converting is finished!


In [14]:
def create_directories_if_not_exist(directories: list):
    # Assert: Is the parameter a list?
    assert isinstance(directories, list), "The parameter must be a list (array)."

    # Assert: Is every element in the list a string?
    for dir_path in directories:
        assert isinstance(dir_path, str), f"The element '{dir_path}' in the list is not a string."

    # Create directories
    for dir_path in directories:
        if not os.path.exists(dir_path):
            os.mkdir(dir_path)
            print(f"Created: {dir_path}")
        else:
            print(f"Already exists: {dir_path}")

In [15]:
import os

# -----------------------------
# Function to safely create dirs
# -----------------------------
def create_directories_if_not_exist(dirs):
    for d in dirs:
        os.makedirs(d, exist_ok=True)
    print("✅ All directories are ready.")


# -----------------------------
# Your dataset structure
# -----------------------------
directories = [
    "/content/drive/MyDrive/zenodo-license/license_plate/dataset/images/train",
    "/content/drive/MyDrive/zenodo-license/license_plate/dataset/images/val",
    "/content/drive/MyDrive/zenodo-license/license_plate/dataset/labels/train",
    "/content/drive/MyDrive/zenodo-license/license_plate/dataset/labels/val"
]

create_directories_if_not_exist(directories)

✅ All directories are ready.


In [16]:
import os
import shutil
import random


def split_images_and_labels(
        val_size = 0.1,
        input_dir = "/content/drive/MyDrive/Project/images",   # ONLY images folder
        labels_dir = "/content/drive/MyDrive/zenodo-license/output/annotations",
        output_dir = "/content/drive/MyDrive/zenodo-license/license_plate/dataset"):

    # ----------------------------
    # Create required directories
    # ----------------------------
    for p in [
        os.path.join(output_dir, "images/train"),
        os.path.join(output_dir, "images/val"),
        os.path.join(output_dir, "labels/train"),
        os.path.join(output_dir, "labels/val")
    ]:
        os.makedirs(p, exist_ok=True)

    # ----------------------------
    # Read all image filenames
    # ----------------------------
    all_images = [
        f for f in os.listdir(input_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    if not all_images:
        raise FileNotFoundError(f"❌ No images found in: {input_dir}")

    random.shuffle(all_images)

    split_idx = int(len(all_images) * (1 - val_size))

    train_imgs = all_images[:split_idx]
    val_imgs   = all_images[split_idx:]


    def copy_split(img_list, split_name):
        for img in img_list:

            # ---------- IMAGE ----------
            src_img = os.path.join(input_dir, img)
            dst_img = os.path.join(output_dir, "images", split_name, img)

            shutil.copy2(src_img, dst_img)

            # ---------- LABEL ----------
            label = os.path.splitext(img)[0] + ".txt"
            src_label = os.path.join(labels_dir, label)
            dst_label = os.path.join(output_dir, "labels", split_name, label)

            if os.path.exists(src_label):
                shutil.copy2(src_label, dst_label)
            else:
                print(f"⚠️  Warning — label missing for: {img}")


    copy_split(train_imgs, "train")
    copy_split(val_imgs, "val")

    print(f"✅ Splitting finished! {len(train_imgs)} train | {len(val_imgs)} val")


# RUN IT
split_images_and_labels()

✅ Splitting finished! 389 train | 44 val


In [17]:
import yaml

data = {
    'path': '/content/drive/MyDrive/zenodo-license/license_plate/dataset',
    'train': 'images/train',
    'val': 'images/val',
    'names': [
        "number_plate"
    ]
}

yaml_path = '/content/drive/MyDrive/zenodo-license/license_plate/dataset/data.yaml'

with open(yaml_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"✅ Data has been written to: {yaml_path}")

✅ Data has been written to: /content/drive/MyDrive/zenodo-license/license_plate/dataset/data.yaml


In [19]:
from ultralytics import YOLO

print("Training on GPU ⚙️")

model = YOLO("yolo11n.pt")   # or yolov8n.pt

results = model.train(
    data="/content/drive/MyDrive/zenodo-license/license_plate/dataset/data.yaml",
    epochs=100,        # reduce for CPU
    imgsz=960,        # reduce for CPU
    batch=16,          # reduce for CPU
    workers=2,
    device=0,     # <<--- IMPORTANT
    patience=20,
    augment=True
)

Training on GPU ⚙️
Ultralytics 8.3.242 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/zenodo-license/license_plate/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_ma

In [5]:
from ultralytics import YOLO

print("Training on GPU ⚙️")

model = YOLO("yolo11s.pt")   #  yolov8 small model

results = model.train(
    data="/content/drive/MyDrive/zenodo-license/license_plate/dataset/data.yaml",
    epochs=100,        # reduce for CPU
    imgsz=960,        # reduce for CPU
    batch=16,          # reduce for CPU
    workers=2,
    device=0,     # <<--- IMPORTANT
    patience=30
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Training on GPU ⚙️
Ultralytics 8.3.243 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/zenodo-license/license_plate/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv

In [9]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
print(model.task)

detect


In [15]:
from ultralytics import settings
print(settings['runs_dir'])

runs


In [18]:
!ls -R runs

ls: cannot access 'runs': No such file or directory
